## Objective of this notebook : illustrate the use of RSA keys to sign and encrypt 

This example is freely inspired from https://medium.com/@Raulgzm/rsa-with-cryptography-python-library-462b26ce4120 .

Maria and Raul want to communicate through an insecure channel. They choose to use RSA system to encryt and signe their messages.

They decide to use the python module Cryptography (see https://cryptography.io ). If this module is not installed, please, run the next cell.

In [1]:
!pip3 install 'cryptography'

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


#### Module loading


First, they need to load the necessary python modules : 

In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.exceptions import InvalidSignature

 #### Key generation
 
 Maria wants to send a message signed and encrypted to Raul. Thus, they decide to generate their pair of RSA keys.

Maria generates her pair of keys.

In [16]:
maria_private_key = rsa.generate_private_key(
 public_exponent=65537,
 key_size=2048,
 backend=default_backend()
)
maria_public_key = maria_private_key.public_key()

#### Key serialization 

To share her public key with other people, she serializes it, i.e. she transforms it as a butes array. Then, we assume that she sends her public key to Raul. 

In [17]:
pem_maria_public_key = maria_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

Raul does the same operations.

In [18]:
raul_private_key = rsa.generate_private_key(
 public_exponent=65537,
 key_size=2048,
 backend=default_backend()
)
raul_public_key = raul_private_key.public_key()

pem_raul_public_key = raul_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

#### Key deserialization

Maria wants to send a message to Raul, so she must use his public key. Thus she first loads it from the serialized form.

In [19]:
loaded_raul_key = load_pem_public_key(pem_raul_public_key) 

#### Encrypting

Now, she can encrypt her message with the Raul's public key. 

In [20]:
message = 'the side must be like a piece of music'
message_bytes = message.encode() 

ciphertext = loaded_raul_key.encrypt(
      message_bytes,
      padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
      )
)

Note : it was shown in the course that the direct encoding of the message with the RSA public key is not secure. So this example is given just to illustrate the use of the keys for the encryption and the signature. In a more secure context, a random number will be generated and will be used to derive the key of a symmetric cipher. This number will be encrypted with the RSA public key and the message will be encrypted with the symmetric cipher and the derived key.

#### Signing

Then, Maria wants to sign her message. Thus, she uses her secret key. 

In [21]:
signature = maria_private_key.sign(
    message_bytes,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

Then, she sents the encrypted message and the signature.

In [22]:
data = (ciphertext, signature)

#### Decrypting

Raul receives the data. First, he must decrypt the ciphertext with his secret key.

In [23]:
plain_text = raul_private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# to have a string version 
plain_text_string = plain_text.decode() 
print(plain_text_string)

the side must be like a piece of music


Then, he verifies the signature with the public key of maria. He first loads the key from the serialized version.

In [13]:
loaded_maria_key = load_pem_public_key(pem_maria_public_key)  

#### Verify the signature

and he verifies the signature.

In [14]:
try : 
    verifier = loaded_maria_key.verify(
        signature,
        plain_text,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
            ),
        hashes.SHA256()
    )
except InvalidSignature :
    print("Invalid signature")